# ADIOS2-Python-Low-Level-MPI: Write Read Global Values and Arrays

## Writer: Generate Global Values and Arrays

In [1]:
from mpi4py import MPI
import numpy as np
import adios2


# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Initialize dimensions
nx = 10
nsteps = 3

# Global array dimensions scattered across MPI nprocs
shape = [size * nx]
start = [rank * nx]
count = [   1 * nx]


# open adios2 file
fw = adios2.open("variable_shapes.bp", "w", comm)

for step in range(0,3):
    
    #global variable that doesn't change
    #save nsteps as a numpy array
    if(step == 0):
        fw.write("nsteps", np.array(nsteps))
        fw.write("size", np.array(size))
    
    #global variable that changes over time
    #save step as a string
    fw.write("step", str(step))
    
    # local values are independent quantities must be explicit
    fw.write("rank", np.array(rank), [adios2.LocalValueDim])
    
    # global array, there is a total shape, 
    # and a local "window" for each rank (start and count)
    fw.write("GlobalArray", np.arange(step, nx, dtype=np.float), shape, start, count, adios2.ConstantDims)
    
    # local array, no shape, no start offset
    # only a local "dimension" for each rank (count)
    fw.write("LocalArray", np.arrange(step, nx, dtype=np.int), [], [], count)
    
# always close if open if not in a with-as instruction
fw.close()

AttributeError: module 'adios2' has no attribute 'LocalValueDim'

In [ ]:
!bpls variable_shapes.bp -lavd -n 10